# Time Series Data Analysis

In [9]:
# Import required libraries.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
filepath = "../../../Data/future_sales/sales_train.csv"
stores_data = pd.read_csv(filepath)

In [13]:
split = StratifiedShuffleSplit()
for train_index, test_index in split.split(stores_data, stores_data['shop_id']):
    print(train_index, test_index)

[1818791 1404583 1281749 ... 1625590   72698  926504] [ 663893  377977 1450758 ... 2095034  420606 2532451]
[2801667  885653 2071390 ...  718872 2227647  988152] [1396722  600078  867041 ...  401772 1344532 1455961]
[1703468 1990754 2413763 ... 1621652  703537  717778] [ 203673 2802054  484685 ... 1543511 2907488 1672604]
[1515857   88354  553106 ... 1580822 2272676  473953] [ 830249 2551054   60695 ...  611639 2264345  928843]
[1042410  357142 2175232 ... 2213776 2380785 2050936] [2312011 2112846 1772053 ... 2637668 1568310 1761234]
[1443230  211697  383853 ... 1023230  670100  594733] [1329818 2709694 2698934 ... 1545138 2215167 1695838]
[2903740 2441049  131496 ... 1900890 1187347 1440376] [2444550 1972850 1142677 ...  609530 1572390 1888069]
[ 649723  200097 2490164 ...  944510 2506721 2179797] [2379259  288902 1621371 ... 2052490 1375428 2076294]
[ 364694 1214819 1485853 ... 2096100 2479095 1968114] [1691338 1046996 1067798 ...  766690 1802812 2218191]
[1816483  942948 1338069 ...

## Quick data exploration
- Want to know the columns and data types
- Basic statistics of the data

In [7]:
stores_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB
